In [2]:
import dotenv, os, requests
import pandas as pd
import numpy as np
import pandasql as ps

In [3]:
dotenv.load_dotenv('ref/.env')

True

In [4]:
league_id = os.environ.get('tds')

In [5]:
lst_rglr_season_wk = 14

In [6]:
base_api_url = 'https://api.sleeper.app/v1/'

In [33]:
league_info = requests.get(f'{base_api_url}league/{league_id}').json()

In [ ]:
league_info = {
    'league_name' : league_info['name']
    ,'start_week' : league_info['settings']['start_week']
    ,'playoff_start_week' : league_info['settings']['playoff_week_start']
    ,'roster_slots' : [pos for pos in league_info['roster_positions'] if pos != 'BN']
}

In [7]:
def get_weekly_matchups_dfs():
    plyr_wkly_points_dicts = []
    wkly_matchups_dicts = []
    wkly_matchups = [requests.get(f'{base_api_url}league/{league_id}/matchups/{wk}').json() for wk in range(league_info['start_week'], league_info['playoff_start_week'])]
    week_num = 0
    for wk in wkly_matchups:
        week_num += 1
        for matchup in wk:
            wkly_matchups_dicts.append({'roster_id' : matchup['roster_id'], 'week' : week_num, 'matchup_id' : matchup['matchup_id'], 'ttl_points' : matchup['points']})
            for player in matchup['players']:
                plyr_wkly_points_dicts.append({'roster_id': matchup['roster_id'], 'player_id' : player, 'week': week_num, 'is_starter' : True if player in matchup['starters'] else False, 'points' : matchup['players_points'][player]})
    return (pd.DataFrame(wkly_matchups_dicts), pd.DataFrame(plyr_wkly_points_dicts))

In [8]:
def get_weekly_transactions_dfs():
    wkly_trnsctns = [requests.get(f'{base_api_url}league/{league_id}/transactions/{wk}').json() for wk in range(league_info['start_week'], league_info['playoff_start_week'])]
    trns_dicts = []
    plyr_trns_dicts = []
    week_num = 0
    player_trns_typs = ['add', 'drop']
    for wk in wkly_trnsctns:
        week_num += 1
        for trns in wk:
            for trns_typ in player_trns_typs:
                if trns[f'{trns_typ}s']:
                    for player in trns[f'{trns_typ}s'].keys():
                        plyr_trns_dicts.append({
                            'week' : week_num
                            ,'player_id' : player
                            ,'roster_id' : trns[f'{trns_typ}s'][player]
                            ,'add_drop' : trns_typ
                            ,'trns_id' : trns['transaction_id']
                        })
            for roster in trns['roster_ids']:
                trns_dicts.append({
                    'roster_id' : roster
                    ,'trns_typ' : trns['type']
                    ,'trns_status' : trns['status']
                    ,'trns_id' : trns['transaction_id']
                })
    return (pd.DataFrame(trns_dicts), pd.DataFrame(plyr_trns_dicts))

In [9]:
def get_roster_id_by_team_df():
    users_df = pd.DataFrame(requests.get(f'{base_api_url}league/{league_id}/users').json())[['display_name', 'user_id']]
    rosters_df = pd.DataFrame([{'roster_id' : roster['roster_id'], 'owner_id' : roster['owner_id'], 'record' : roster['metadata']['record']} for roster in requests.get(f'{base_api_url}league/{league_id}/rosters').json()])
    return users_df.merge(rosters_df, how='inner', left_on='user_id', right_on='owner_id')[['display_name', 'roster_id', 'record']]


In [10]:
matchups_df, plyr_points_df = get_weekly_matchups_dfs()

In [11]:
trns_df, plyr_trns_df = get_weekly_transactions_dfs()

In [12]:
plyr_df = pd.read_json('ref/players.json', orient='index')[['first_name', 'last_name', 'team', 'position', 'player_id', 'years_exp']]

/var/folders/0b/_drclt_54kb8why6nm6znsnh0000gn/T/ipykernel_55124/2337021972.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  plyr_df = pd.read_json('ref/players.json', orient='index')[['first_name', 'last_name', 'team', 'position', 'player_id', 'years_exp']]
/var/folders/0b/_drclt_54kb8why6nm6znsnh0000gn/T/ipykernel_55124/2337021972.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  plyr_df = pd.read_json('ref/players.json', orient='index')[['first_name', 'last_name', 'team', '

In [13]:
rosters_df = get_roster_id_by_team_df()

In [ ]:
def get_rookie_starts_df(plyr_points_df, plyr_df, rosters_df):
    strted_df = plyr_points_df[plyr_points_df.is_starter == True].merge(plyr_df[plyr_df.years_exp == 0], on='player_id', how='inner').merge(rosters_df, on='roster_id', how='inner')[['player_id', 'display_name', 'week']]
    rookies_strtd_df = strted_df.groupby('display_name').player_id.nunique().to_frame().reset_index().rename({'player_id' : 'num_rookies_started'}, axis=1)
    rookie_strts_df = strted_df.groupby('display_name').player_id.count().to_frame().reset_index().rename({'player_id' : 'ttl_rookie_starts'}, axis=1)
    return rookies_strtd_df.merge(rookie_strts_df).sort_values(by=['ttl_rookie_starts', 'num_rookies_started'], ascending=False)

In [ ]:
def get_win_streak(row):
    return max(map(len, row.split('L')))

def get_loss_streak(row):
    return max(map(len, row.split('W')))

In [ ]:
rosters_df['win_streak'] = rosters_df.record.apply(get_win_streak)
rosters_df['loss_streak'] = rosters_df.record.apply(get_loss_streak)

In [20]:
ps.sqldf("""
SELECT
    display_name
    ,SUM(points) AS pf
    ,SUM(points) / 14 AS wkly_avg_pf
    FROM plyr_points_df ppdf
    INNER JOIN rosters_df rdf
        ON rdf.roster_id = ppdf.roster_id
    WHERE 1=1
        AND is_starter = True
    GROUP BY 1
    ORDER BY 2 DESC
""")

,display_name,pf,wkly_avg_pf
0,aspear3,2024.04,144.574286
1,hatersandlosers,2017.32,144.094286
2,kerrveball,1982.44,141.602857
3,rmasons,1883.40,134.528571
4,UB400,1769.20,126.371429
5,MadelynShaw,1726.32,123.308571
6,blspain003,1654.56,118.182857
7,IWantSomeButts,1653.74,118.124286
8,GriffDaGreat,1587.60,113.400000
9,catrussell,1569.76,112.125714


In [21]:
f'{base_api_url}league/{league_id}/users'

'https://api.sleeper.app/v1/league/1048281947404955648/users'

In [22]:
rosters_df

,display_name,roster_id,record
0,rmasons,1,WLLWWLLWWWWLLW
1,ItIsATravisty,11,LLWWLLWLLLLLWL
2,aspear3,8,WWWLWLWLLWWWWW
3,kerrveball,7,LWLWWWWWWLWWLW
4,catrussell,3,WLLLLWLLWWLWLL
5,hatersandlosers,2,WWWWWWLWWLWLWW
6,IWantSomeButts,4,WLLLLWLWLLWLLW
7,MadelynShaw,5,WWWWWLWLWWWWWL
8,GriffDaGreat,6,LLLLLLLWWWLWLL
9,RyJohn,9,LLLLLWLLLLLLLW


In [23]:
tst_rcrd = 'LWWWWLWWLLLWWL'

In [25]:
rosters_df

,display_name,roster_id,record
0,rmasons,1,WLLWWLLWWWWLLW
1,ItIsATravisty,11,LLWWLLWLLLLLWL
2,aspear3,8,WWWLWLWLLWWWWW
3,kerrveball,7,LWLWWWWWWLWWLW
4,catrussell,3,WLLLLWLLWWLWLL
5,hatersandlosers,2,WWWWWWLWWLWLWW
6,IWantSomeButts,4,WLLLLWLWLLWLLW
7,MadelynShaw,5,WWWWWLWLWWWWWL
8,GriffDaGreat,6,LLLLLLLWWWLWLL
9,RyJohn,9,LLLLLWLLLLLLLW


In [28]:
ps.sqldf("""
SELECT
    display_name
    ,win_streak
    FROM rosters_df
    WHERE 1=1
    ORDER BY 2 DESC         
""")

,display_name,win_streak
0,kerrveball,6
1,hatersandlosers,6
2,aspear3,5
3,MadelynShaw,5
4,rmasons,4
5,blspain003,4
6,GriffDaGreat,3
7,ItIsATravisty,2
8,catrussell,2
9,UB400,2


In [29]:
ps.sqldf("""
SELECT
    display_name
    ,loss_streak
    FROM rosters_df
    WHERE 1=1
    ORDER BY 2 DESC
""")

,display_name,loss_streak
0,GriffDaGreat,7
1,RyJohn,7
2,ItIsATravisty,5
3,catrussell,4
4,IWantSomeButts,4
5,blspain003,3
6,rmasons,2
7,aspear3,2
8,UB400,2
9,kerrveball,1


In [72]:
plyr_trns_df[plyr_trns_df.player_id == '5937']

,week,player_id,roster_id,add_drop,trns_id
1,1,5937,7,drop,1139286557552222208
124,3,5937,10,add,1142721650241306624
265,6,5937,1,add,1151734534112149504
269,6,5937,10,drop,1151734534112149504
406,8,5937,12,add,1155164738595229696
409,8,5937,1,drop,1155164738595229696
558,12,5937,12,drop,1167135238015856640


In [31]:
trns_df

,roster_id,trns_typ,trns_status,trns_id
0,7,waiver,complete,1139286557552222208
1,1,waiver,complete,1139274194702811136
2,11,waiver,complete,1139238178269347840
3,11,waiver,complete,1139237838631124992
4,11,waiver,complete,1139237558925385728
...,...,...,...,...
354,11,free_agent,complete,1170030050385711104
355,11,free_agent,complete,1170029645354356736
356,11,free_agent,complete,1170029217111724032
357,10,free_agent,complete,1170007348002279424


In [75]:
plyr_eff_dts = ps.sqldf("""
SELECT
    l.roster_id
    ,l.player_id
    ,pdf.first_name
    ,pdf.last_name
    ,l.week AS eff_fm_wk
    ,r.week AS eff_to_wk
    FROM plyr_trns_df l
    LEFT JOIN plyr_trns_df r
        ON l.roster_id = r.roster_id
        AND l.player_id = r.player_id
        AND l.week < r.week
    INNER JOIN plyr_df pdf
        ON pdf.player_id = l.player_id
    WHERE 1=1
        AND l.add_drop = 'add'
        AND r.add_drop = 'drop'
""")

In [60]:
ps.sqldf("""
SELECT
    pdf.player_id
    ,pdf.first_name
    ,pdf.last_name
    ,ptdf.add_drop
    ,ptdf.trns_id
    ,ptdf.roster_id
    ,tdf.trns_typ
    FROM plyr_df pdf
    INNER JOIN plyr_trns_df ptdf
        ON ptdf.player_id = pdf.player_id
    INNER JOIN trns_df tdf
        ON tdf.trns_id = ptdf.trns_id
        AND tdf.trns_typ IN ('waiver', 'free_agent')
    WHERE 1=1
        AND ptdf.add_drop = 'add'
"""
)

,player_id,first_name,last_name,add_drop,trns_id,roster_id,trns_typ
0,2197,Brandin,Cooks,add,1139286557552222208,7,waiver
1,8134,Khalil,Shakir,add,1139274194702811136,1,waiver
2,5987,Alexander,Mattison,add,1139238178269347840,11,waiver
3,1945,Chris,Boswell,add,1139237838631124992,11,waiver
4,LAC,Los Angeles,Chargers,add,1139237558925385728,11,waiver
...,...,...,...,...,...,...,...
272,9508,Tyjae,Spears,add,1170030050385711104,11,free_agent
273,TEN,Tennessee,Titans,add,1170029645354356736,11,free_agent
274,9999,Will,Levis,add,1170029217111724032,11,free_agent
275,7839,Evan,McPherson,add,1170007348002279424,10,free_agent


In [49]:
ps.sqldf("""
SELECT
    display_name
    ,first_name
    ,last_name
    ,SUM(CASE
        WHEN is_starter = True
            THEN points
        ELSE 0
    END) as strtr_points
    ,SUM(points) AS ttl_points
    FROM rosters_df rdf
    INNER JOIN plyr_points_df ppdf
        ON ppdf.roster_id = rdf.roster_id
    INNER JOIN plyr_df pdf
        ON pdf.player_id = ppdf.player_id
    INNER JOIN plyr_trns_df ptdf
        ON ptdf.roster_id = rdf.roster_id
        AND ptdf.add_drop = 'add'
    INNER JOIN trns_df tdf
        ON tdf.trns_id = ptdf.trns_id
        AND tdf.roster_id = rdf.roster_id
        AND tdf.trns_typ IN ('waiver', 'free_agent')
    GROUP BY 1,2,3
"""
)

,display_name,first_name,last_name,strtr_points,ttl_points
0,GriffDaGreat,Amon-Ra,St. Brown,714.8,714.8
1,GriffDaGreat,Breece,Hall,715.0,715.0
2,GriffDaGreat,Brock,Purdy,1874.0,1874.0
3,GriffDaGreat,Cade,Otton,74.0,169.0
4,GriffDaGreat,Cincinnati,Bengals,17.0,151.0
...,...,...,...,...,...
397,rmasons,Tyreek,Hill,1040.4,2818.8
398,rmasons,Washington,Commanders,187.2,187.2
399,rmasons,Will,Dissly,1101.6,1101.6
400,rmasons,Zach,Ertz,0.0,259.2


In [91]:
ps.sqldf("""
SELECT
    display_name
    ,count(tdf.trns_id) AS waivers
    FROM trns_df tdf
    INNER JOIN rosters_df rdf
        ON rdf.roster_id = tdf.roster_id
    WHERE 1=1
        AND tdf.trns_status = 'complete'
        AND (tdf.trns_typ = 'waiver'
            OR tdf.trns_typ = 'free_agent')
    GROUP BY 1
    ORDER BY 2 DESC
""")

,display_name,waivers
0,UB400,37
1,rmasons,35
2,kerrveball,30
3,ItIsATravisty,29
4,IWantSomeButts,28
5,catrussell,24
6,RyJohn,22
7,blspain003,19
8,hatersandlosers,18
9,MadelynShaw,17


In [48]:
ps.sqldf("""
SELECT
    display_name
    ,count(distinct tdf.trns_id) AS trades
    FROM trns_df tdf
    INNER JOIN rosters_df rdf
        ON rdf.roster_id = tdf.roster_id
    WHERE 1=1
        AND tdf.trns_status = 'complete'
        AND tdf.trns_typ = 'trade'
    GROUP BY 1
    ORDER BY 2 DESC
""")

,display_name,trades
0,rmasons,10
1,kerrveball,5
2,hatersandlosers,5
3,catrussell,5
4,GriffDaGreat,3
5,UB400,2
6,RyJohn,2
7,blspain003,1
8,MadelynShaw,1
9,ItIsATravisty,1


In [45]:
rosters_df.sort_values('roster_id')

,display_name,roster_id,record,win_streak,loss_streak
0,rmasons,1,WLLWWLLWWWWLLW,4,2
5,hatersandlosers,2,WWWWWWLWWLWLWW,6,1
4,catrussell,3,WLLLLWLLWWLWLL,2,4
6,IWantSomeButts,4,WLLLLWLWLLWLLW,1,4
7,MadelynShaw,5,WWWWWLWLWWWWWL,5,1
8,GriffDaGreat,6,LLLLLLLWWWLWLL,3,7
3,kerrveball,7,LWLWWWWWWLWWLW,6,1
2,aspear3,8,WWWLWLWLLWWWWW,5,2
9,RyJohn,9,LLLLLWLLLLLLLW,1,7
10,UB400,10,LWWLLWWLLWLLWL,2,2


In [46]:
ps.sqldf("""
SELECT
    min(l.roster_id) as rst_one
    ,max(r.roster_id) as rst_two
    ,l.trns_id
    FROM trns_df l
    INNER JOIN trns_df r
        ON r.trns_id = l.trns_id
        AND l.roster_id <> r.roster_id
    WHERE 1=1
        AND l.trns_typ = 'trade'
    GROUP BY 3
    ORDER BY 1 ASC, 2 ASC
""")

,rst_one,rst_two,trns_id
0,1,2,1151945073312104448
1,1,3,1131683069649408000
2,1,3,1149904918766047232
3,1,3,1161426611557601280
4,1,4,1149887513054691328
5,1,5,1113668412531654656
6,1,6,1048338653581361152
7,1,9,1154462251492069376
8,1,10,1151734534112149504
9,1,12,1155164738595229696
